## `setup`

In [1]:
from abc import ABC, abstractmethod

# Introduction

This notebook presents Python implementations of functors, applicative functors and modads. These concepts are well-known in the context of functional programming, especially in the Haskell community.

## Functor

A **functor** is an interface for types that are mappable.

The interface has one operation, `fmap`, that given an object (containing values of type `a`), and a function `a->b`, returns another object (containing values of type `b`).

In Python, we can define this interface like this,

In [2]:
class Functor(ABC):
  @abstractmethod
  def fmap(self, f): # fmap :: f a -> (a -> b) -> f b
    ...

  ## free dunder: a | b == a.fmap(b)
  ## dunder needs to be a left associative operator (eg, >= does not work)
  def __or__(self, f):
    return self.fmap(f)

  def __repr__(self):
    return str(self.__class__.__name__)

A well-known example of functor is `Maybe` (aka `Optional`) that represents an optional value. A value of this type might or might not hold information. We'll define two subclasses `Just` and `Nothing` to represent these two possibilities:

In [3]:
class Maybe(Functor):
  ...

For `Nothing`, regardeless of the `Maybe` value received, the result is still `Nothing`,

In [4]:
class Nothing(Maybe):
  def fmap(self, f):
    return self

For `Just`, when receiving a `Maybe` value, it applies function `f` to the associated value, and return a `Just` value with that result. If something goes wrong it returns a `Nothing` instead,

In [5]:
class Just(Maybe):
  def __init__(self, x):
    self.x = x

  def fmap(self, f):
    try:
      return Just(f(self.x))
    except:
      return Nothing()

  def __repr__(self):
    return f'Just({str(self.x)})'

This functor is also known as `Optional` in languages like Java.

Some use cases:

In [7]:
print(Just(2).fmap(bool))
print(Just(2).fmap(lambda x:2*x))
print(Just(8).fmap(lambda x:1/x).fmap(lambda x:100*x))
print(Just(0).fmap(lambda x:1/x).fmap(lambda x:100*x))

Just(True)
Just(4)
Just(12.5)
Nothing


As seen in the last examples, we can chain several `fmap` applications,

In [ ]:
print(Just([7,2,16]).fmap(len)
                    .fmap(lambda n:range(n*2))
                    .fmap(sum)
     )

# using | instead
print(Just([7,2,16]) | len | (lambda n:range(n*2)) | sum)

Just(15)
Just(15)


Another example (seen [here](https://www.youtube.com/watch?v=e6tWJD5q8uw)): we receive a _string_ with two integers comma separated. We wish to parse the numbers and return their integer division.

A tradicional approach is,

In [ ]:
def split(xs):
  return tuple(xs.split(','))

def parse(p):
  return int(p[0]), int(p[1])

def divide(p):
  a,b = p
  return a//b

print(divide(parse(split('34,10'))))

3


With an invalid input, an exceptions occurs,

In [ ]:
print(divide(parse(split('34,1a')))) # an exception is raised

ValueError: invalid literal for int() with base 10: '1a'

The next implementation uses functor `Maybe` to do the same task.

If some error occurs, the computation will produce value `Nothing`, i.e., it automatically deals with possible exceptions (something that we didn't have in the tradicional approach).

In [ ]:
print(Just('34,10') | split | parse | divide)
print(Just('34,1a') | split | parse | divide)

Just(3)
Nothing


Another functor is the list datatype. Here `fmap` is the `map` operation, applying the given function over each list element,

In [8]:
class List(Functor):
  def __init__(self, *vals):
    self.val = list(vals)

  def fmap(self, f):
    return List(*map(f, self.val))

  def __repr__(self):
    return f'List({str(self.val)})'

In [9]:
print(List(1,0,2) | bool)
print(List(1,2,3) | (lambda x:1.5*x))

List([True, False, True])
List([1.5, 3.0, 4.5])


Another famous example is `Either` where we can have two types of possible values. In the next class we will keep, on `Right` values, the current computation state; and in `Left` values an (eventual) error report.

In [ ]:
class Either(Functor):
  pass

In [ ]:
class Left(Either):
  def __init__(self, error):
    self.error = error

  def fmap(self, f):
    return Left(self.error)

  def __repr__(self):
    return f'Left({self.error})'

In [ ]:
class Right(Either):
  def __init__(self, val=None):
    self.val = val

  def fmap(self, f):
    try:
      return Right(f(self.val))
    except Exception as e:   # error, turn Right into Left
      return Left(e.args[0]) # and save the error msg

  def __repr__(self):
    return f'Right({self.val})'

The same use case, now with functor `Either`, identifies the type of error, when it happens,

In [ ]:
print(Right('34,10') | split | parse | divide)
print(Right('34;10') | split | parse | divide)
print(Right('34,a0') | split | parse | divide)
print(Right('34, 0') | split | parse | divide)

Right(3)
Left(invalid literal for int() with base 10: '34;10')
Left(invalid literal for int() with base 10: 'a0')
Left(integer division or modulo by zero)


The next robust program asks for two integers and divides them,

In [ ]:
print(Right()
       .fmap(lambda _:     int(input("Insert number a: ")))
       .fmap(lambda x: x / int(input("Insert number b: ")))
     )

Insert number a: 20
Insert number b: a
Left(invalid literal for int() with base 10: 'a')


Considering this specific implementation, all functors are objects, we could include extra information about the object state.

The next example keeps the time taken to run each operation,

In [ ]:
import time

class Timer(Functor):
  def __init__(self, value=None, report=None):
    self.value = value
    self.report = report if report is not None else []

  def fmap(self, f):
    t0 = time.time()
    output = f(self.value)
    t1 = time.time()
    self.report.append(f'{f.__name__}: {1e6*(t1-t0):.2f} μs')
    return Timer(output, self.report)

  def state(self):
    return '\n'.join(self.report)

  def __repr__(self):
    return str(self.value)

In [ ]:
a = Timer('34,10') | split | parse | divide
print(a.state())
print(a)

split: 6.20 μs
parse: 6.20 μs
divide: 2.15 μs
3


Class `Timer` is not just a functor. Usually functors do not have extra state. However, the idea of keeping state with a functor can be integrated into the notion of _monad_, as we will see below.

There are functor laws, i.e., semantic laws the programmer must be sure are respected by the functor types:

+ `fmap id == id`, if we map using the identity function, the initial result is not changed

In [ ]:
print(Just(200).fmap(lambda x:x))

Just(200)


+ `fmap (f.g) == fmap f . fmap g`, it does not matter if we apply the composition of two function, or map one after the other

In [ ]:
compose = lambda f,g: lambda x: f(g(x))

f = lambda x: str(x)+'!'
g = lambda x: x+100

print(Just(200).fmap(g).fmap(f))
print(Just(200).fmap(compose(f,g)))

Just(300!)
Just(300!)


## Applicative Functor

In the examples above, we are just mapping unary functions.

If we want to deal with function with two or more parameters, we could use partial functions.

One example,

In [10]:
from functools import partial
from operator import mul

print(Just(5).fmap(partial(mul,3)))

print(List(1,2,3)
        .fmap(lambda x: partial(mul,x))
        .fmap(lambda f: f(10))
     )

Just(15)
List([10, 20, 30])


But there are limitations. We cannot start with a function, `fmap` is unable to process it,

In [11]:
Just(partial(mul, 3)).fmap(5)

Nothing

To deal with this, we introduce a new interface, the **applicative functor**.

There are two operations:

+ `pure`: given a value, returns an applicative functor containing the value (some sources call this _lifting the value_)

+ `apply`: given a functor with `f` and a functor with `x`, returns a functor with `map(f,x)`

The `pure` operation lifts a value into the context of the applicative. The relation with the functor's operation `fmap` is then,

+ `fmap(f, x) == pure(f) >> x`, being `>>` the infix operator for `apply`

The applicative laws:

+ `pure(id) >> x == x`

+ `pure(.) >> f >> g >> h == f >> (g >> h)`, where `.` is composition

The interface in Python:

In [12]:
class Applicative(Functor):
  @classmethod
  def pure(cls, val):
    return cls(val)

  @abstractmethod
  def apply(self, x): # haskell (<*>) :: f (a -> b) -> f a -> f b
    ...

  # free dunder
  def __rshift__(self, x):  # a >> b === a.apply(b)
    return self.apply(x)

Since we need to apply values partially, the next helper function `is_partial` checks if a given function is partial, i.e., if there are some parameters left to assign,

In [ ]:
import inspect

# ref: https://stackoverflow.com/questions/53201023
def is_partial(f):
    signature = inspect.signature(f.func)
    try:
       signature.bind(*f.args, **f.keywords)
       return False
    except TypeError:
       return True

In [ ]:
from functools import partial
from operator import mul

assert     is_partial(partial(mul,1))
assert not is_partial(partial(mul,1,2))

Let's check a refactoring of `Maybe` as an applicative functor:

In [ ]:
class Maybe(Applicative):
  ...

In [ ]:
class Nothing(Maybe):
  def fmap(self, f):
    return self

  def apply(self, xs):
    return self

In [ ]:
class Just(Maybe):
  def __init__(self, x):
    self.x = x

  def fmap(self, f):
    try:
      f = partial(f, self.x)
      return Just(f) if is_partial(f) else Just(f())
    except:
      return Nothing()

  def apply(self, rhs):
    f, self.x = self.x, rhs
    return self.fmap(f)

  def __repr__(self):
    return f'Just({str(self.x)})'

The previous example now works,

In [ ]:
print(Just(partial(mul,3)) >> 5)

Just(15)


`mul` does no need to be partially defined, we can assign each parameter by successive calls to `apply`

In [ ]:
from operator import add, floordiv # https://docs.python.org/3/library/operator.html

print(Just(add) >> 15 >> 20)
print(Just(floordiv) >> 4 >> 0)

print(Just(lambda x,y,z: x+10*y+100*z) >> 1 >> 2 >> 3)

Just(35)
Nothing
Just(321)


Let's make lists as applicative functors. `pure` lifts `x` returning `List(x)`.

For `apply` we need to map each function (of a possible list of function) over all elements from the next applicative value,

In [ ]:
class List(Applicative):
  def __init__(self, val):
    self.val = val

  @classmethod
  def pure(cls, *val):
    return List(list(val))

  def fmap(self, f):
    return List([f(x) for x in self.val])

  def apply(self, xs):
    return List([self._apply(f,x) for f in self.val for x in xs.val])

  def _apply(self, f, x):
    f = partial(f, x)
    return f if is_partial(f) else f()

  def __repr__(self):
    return f'List({str(self.val)})'

This behavior is similar to compreension lists:

In [ ]:
from operator import mul, add

a = List.pure(add, mul)
x = List.pure(1,20,75)

a >> x >> x

List([2, 21, 76, 21, 40, 95, 76, 95, 150, 1, 20, 75, 20, 400, 1500, 75, 1500, 5625])

We can combine `fmap` and `apply`:

In [ ]:
(a >> x >> x).fmap(lambda x: partial(add,x)) >> List.pure(1000,2000)

List([1002, 2002, 1021, 2021, 1076, 2076, 1021, 2021, 1040, 2040, 1095, 2095, 1076, 2076, 1095, 2095, 1150, 2150, 1001, 2001, 1020, 2020, 1075, 2075, 1020, 2020, 1400, 2400, 2500, 3500, 1075, 2075, 2500, 3500, 6625, 7625])

The next example emulates a compreension list,

In [ ]:
print([x*y for x in [2,3,4] for y in [10,100,1000]])

print(List.pure(mul) >> List.pure(2,3,4)
                     >> List.pure(10,100,1000)
     )

[20, 200, 2000, 30, 300, 3000, 40, 400, 4000]
List([20, 200, 2000, 30, 300, 3000, 40, 400, 4000])


There is another interpretation for a list applicative, `ZipList`, which has a behavior similar to `zip`,

In [ ]:
from itertools import cycle

class ZipList(Applicative):
  def __init__(self, val):
    self.val = val

  @classmethod
  def pure(cls, *val):
    return ZipList(cycle(val))

  def fmap(self, f):
    return ZipList([self._apply(f,x) for x in self.val])

  def apply(self, xs):
    return ZipList([self._apply(f,x) for f,x in zip(self.val, xs)])

  def _apply(self, f, x):
    f = partial(f, x)
    return f if is_partial(f) else f()

  def __iter__(self):
    return iter(self.val)

  def __repr__(self):
    return f'List({str(self.val)})'

In [ ]:
# multiplies or sums, alternatively, pairs of values
ZipList.pure(mul, add) >> ZipList(range(10)) >> ZipList(range(10))

List([0, 2, 4, 6, 16, 10, 36, 14, 64, 18])

## Monads

A **monad** is an interface with the following operations:

+ `unit` lifts a value to a monad value (like `pure`), aka as `return`

+ `bind` applies a given function `f` to the modad value, returning a new monad value

However, the function `f` given to `bind` is not of type `a -> b`, like in `fmap`, but of type `a -> monad[b]`. That is, `f` is able to create monadic values.

The interface in Python:

In [13]:
# we could subclass from Applicative,
# but choosing ABC for brevity, so to not (re)implement fmap nor apply
class Monad(ABC):
  @classmethod
  def unit(cls, value):
    return cls(value)

  @abstractmethod
  def bind(self, mf):  # Haskell (>>=) :: m a -> (a -> m b) -> m b
    ...

  # free dunder
  def __add__(self, mf):   # a + b === a.bind(b)
    return self.bind(mf)

  def __repr__(self):
    return str(self.__class__.__name__)

Let's make `Maybe` a monad:

In [ ]:
class Maybe(Monad):
  ...

In [ ]:
class Nothing(Maybe):
  def bind(self, mf):
    return self

In [ ]:
class Just(Maybe):
  def __init__(self, value):
    self.value = value

  def bind(self, mf):
    result = mf(self.value)
    if isinstance(result, Maybe): # mf :: a -> m b
      return result
    else:                         # extra behavior, accept
      return Just(result)         # mf :: a -> b (ie, does fmap)

  def __repr__(self):
    return f'Just({str(self.value)})'

Monads deal with functions that produce monadic values:

In [ ]:
dec = lambda n: Just(n-1) if n>=1 else Nothing()

print(Just(3).bind(dec))
print(Just(0) + dec)
print(Nothing() + dec)
print(Just(3) + dec + dec)

Just(2)
Nothing
Nothing
Just(1)


Adapting the functions from the parsing example above:

In [ ]:
def split(xs):
  try:
    tokens = xs.split(',')
    return Just(tuple(tokens))
  except:
    return Nothing

def parse(p):
  try:
    x,y = p
    return Just( (int(x), int(y)) )
  except:
    return Nothing

def divide(p):
  try:
    a,b = p
    return Just(a//b)
  except:
    return Nothing

In [ ]:
Just('34,10') + split + parse + divide

Just(3)

Let's now make list a monad:

In [14]:
from itertools import chain
concat = chain.from_iterable

class List(Monad):
  def __init__(self, *vals):
    self.val = list(vals)

  def bind(self, mf):
    return List(*concat(map(mf, self.val)))
   #return List(*[x for xs in map(mf, self.value) for x in xs])

  def __iter__(self):
    return iter(self.val)

  def __repr__(self):
    return str(self.val)

Some use cases (notice that funtions now return `List` values):

In [15]:
List(3,4,5) + (lambda x: List(-x,x))

[-3, 3, -4, 4, -5, 5]

In [ ]:
List(3,4,5) + (lambda x: List(*[(x,c) for c in 'ab']))

[(3, 'a'), (3, 'b'), (4, 'a'), (4, 'b'), (5, 'a'), (5, 'b')]

In [ ]:
from functools import partial
pair = lambda x,y: (x,y)

print(List(3,4,5)
        + (lambda n: List(partial(pair,n)))
        + (lambda f: List(*[f(c) for c in 'ab'])))

[(3, 'a'), (3, 'b'), (4, 'a'), (4, 'b'), (5, 'a'), (5, 'b')]


Every monad is a functor and an applicative functor.

For eg, we can define `fmap` as `fmap(f, xs) == xs >>= unit . f`,

In [18]:
compose = lambda f,g: lambda x: f(g(x))
inc = lambda x: x+1

List(1,2,3) + compose(List.unit, inc)  # same as fmap(inc, [1,2,3])

[2, 3, 4]

Monad also have semantic laws:

+ `m >>= unit == m`

+ `unit x >>= f == f(x)`, identity laws saying `unit` does not perform computations

In [19]:
List(1,2,3) + List.unit

[1, 2, 3]

In [41]:
inc_m = lambda x: List(x+1)

print(List.unit(10) + inc_m , inc_m(10)) # == not implemented, just print it

[11] [11]


+ `(m >>= f) >>= g == m >>= (lambda x: f(x) >>= g)`, the associative law

In [35]:
double_m = lambda x: List(2*x)
value = 10
m = List.unit(value)

(m + inc_m) + double_m

[22]

In [36]:
(lambda x: inc_m(x) + double_m)(value)

[22]

Let's check another example: reimplementing `Timer` as a monad,

In [ ]:
class Timer(Monad):
  def __init__(self, value=None, report=None):
    self.value = (value, report if report is not None else [])

  def bind(self, mf):
    return mf(self.value)

  def state(self):
    return '\n'.join(self.value[1])

  def __repr__(self):
    return str(self.value[0])

The functions will now have the responsability of dealing with state (in this case, timing how much time it takes the computation):

In [ ]:
import time

def split(state):
  value, report = state
  t0 = time.time()
  tokens = value.split(',')
  t1 = time.time()
  report.append(f'split: {1e6*(t1-t0):.2f} μs')
  return Timer(tokens, report)

def parse(state):
  value, report = state
  t0 = time.time()
  p = int(value[0]), int(value[1])
  t1 = time.time()
  report.append(f'parse: {1e6*(t1-t0):.2f} μs')
  return Timer(p, report)

def divide(state):
  value, report = state
  t0 = time.time()
  a,b = value
  t1 = time.time()
  report.append(f'divide: {1e6*(t1-t0):.2f} μs')
  return Timer(a//b, report)

In [ ]:
a = Timer('34,10') + split + parse + divide
print(a.state())
print(a)

split: 1.91 μs
parse: 3.34 μs
divide: 0.24 μs
3


This solution seems a stepback from the previous `Timer` functor. The code became more repetitive!

Notice however, that `fmap` will not always have the needed information to update the object state. The only one that has access to all relevant information is the function itself. So, the monad will be able to solve problems that the functor (even with the help of some extra attributes) cannot.

Let's add some more details to the report: besides the execution time, let's also include the current step of processing the initial string, and which operation is executing at each step:

In [ ]:
import time

def split(state):
  value, report = state
  t0 = time.time()
  tokens = value.split(',')
  t1 = time.time()
  report.append(f'split {value} --> {tokens}: {1e6*(t1-t0):.2f} μs')
  return Timer(tokens, report)

def parse(state):
  value, report = state
  t0 = time.time()
  p = int(value[0]), int(value[1])
  t1 = time.time()
  report.append(f'parse {value} --> {p}: {1e6*(t1-t0):.2f} μs')
  return Timer(p, report)

def divide(state):
  value, report = state
  t0 = time.time()
  a,b = value
  t1 = time.time()
  report.append(f'divide {a} by {b}: {1e6*(t1-t0):.2f} μs')
  return Timer(a//b, report)

In [ ]:
a = Timer('34,10') + split + parse + divide
print(a.state())
print(a)

split 34,10 --> ['34', '10']: 2.15 μs
parse ['34', '10'] --> (34, 10): 5.96 μs
divide 34 by 10: 0.72 μs
3


## References

+ Miran Lipovaca, [Learn You a Haskell for Great Good!](http://learnyouahaskell.com/chapters), caps 11--13

+ Jeremy Bowers, [Functors and Monads For People Who Have Read Too Many "Tutorials"](https://www.jerf.org/iri/post/2958/)

+ César Tron-Lozai, [No Nonsense Monad & Functor](https://www.youtube.com/watch?v=e6tWJD5q8uw)